In [ ]:
import sqlite3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
con = sqlite3.connect("/content/drive/MyDrive/sql/classic.db")

1. Calculate the average revenue for each country in 2004.

In [ ]:
q = """
SELECT c.country, AVG(od.quantityOrdered * od.priceEach) AS avg_revenue
FROM orderDetails od
INNER JOIN orders o ON o.orderNumber = od.orderNumber
INNER JOIN customers c ON c.customerNumber = o.customerNumber
WHERE strftime('%Y', o.orderDate) = '2004'
GROUP BY c.country
"""

pd.read_sql(q, con)

,country,avg_revenue
0,Australia,3141.741231
1,Austria,4102.805833
2,Belgium,3231.972609
3,Canada,2682.696304
4,Denmark,3249.473636
5,Finland,3588.555000
6,France,3108.343620
7,Germany,3264.187805
8,Ireland,3118.641875
9,Italy,3142.252632


2. List the customers who have placed orders for products from at least two different categories (productline).

In [ ]:
q = """
SELECT *
FROM (
SELECT c.customerNumber, COUNT(DISTINCT p.productLine) AS distinct_productLine
FROM customers c
INNER JOIN orders o ON c.customerNumber = o.customerNumber
INNER JOIN orderDetails od ON o.orderNumber = od.orderNumber
INNER JOIN products p ON od.productCode = p.productCode
GROUP BY c.customerNumber)
WHERE distinct_productLine >= 2
"""

pd.read_sql(q, con)

,customerNumber,distinct_productLine
0,103,3
1,112,2
2,114,5
3,119,6
4,121,5
...,...,...
92,486,3
93,487,3
94,489,4
95,495,4


3. Find the supplier that provides the most products for each product category. Include the category, supplier name, and the number of products supplied.

In [ ]:
q = """
SELECT productLine AS product_category, productVendor AS supplier_name, num_products
FROM (
SELECT productLine, productVendor, COUNT(productCode) AS num_products,
ROW_NUMBER() OVER (PARTITION BY productLine ORDER BY COUNT(productCode) DESC) AS rank
FROM products p
GROUP BY productLine, productVendor)
WHERE rank = 1
"""

pd.read_sql(q, con)

,product_category,supplier_name,num_products,rank
0,Classic Cars,Classic Metal Creations,6,1
1,Motorcycles,Highway 66 Mini Classics,3,1
2,Planes,Motor City Art Classics,3,1
3,Ships,Unimax Art Galleries,2,1
4,Trains,Gearbox Collectibles,1,1
5,Trucks and Buses,Motor City Art Classics,2,1
6,Vintage Cars,Exoto Designs,4,1


4. Calculate the average amount invoiced per month during 2003.

In [ ]:
q = """
SELECT strftime('%m-%Y', o.orderDate) AS month, AVG(p.amount) AS avg_amount
FROM orders o
INNER JOIN customers c ON c.customerNumber = o.customerNumber
INNER JOIN payments p ON p.customerNumber = c.customerNumber
WHERE strftime('%Y', o.orderDate) = '2003'
GROUP BY month
"""

pd.read_sql(q, con)

,month,avg_amount
0,01-2003,40179.572593
1,02-2003,34261.613000
2,03-2003,43834.105652
3,04-2003,28141.733333
4,05-2003,29702.561000
5,06-2003,44560.296744
6,07-2003,39655.586207
7,08-2003,41498.999091
8,09-2003,37885.495429
9,10-2003,35551.240339


5. List all customers who have not placed any orders in 2005. Show the customer's name and the date of their last order.

In [ ]:
q = """
SELECT customerName, orderDate as last_orderDate
FROM (
SELECT o.customerNumber, c.customerName, o.orderDate,
ROW_NUMBER() OVER (PARTITION BY o.customerNumber ORDER BY o.orderDate DESC) AS rank
FROM orders o
LEFT JOIN customers c ON c.customerNumber = o.customerNumber
WHERE strftime('%Y', orderDate) != '2005'
GROUP BY o.customerNumber)
"""

pd.read_sql(q,con)

,customerName,last_orderDate
0,Atelier graphique,2003-05-20
1,Signal Gift Stores,2003-05-21
2,"Australian Collectors, Co.",2003-04-29
3,La Rochelle Gifts,2004-07-23
4,Baane Mini Imports,2003-01-29
...,...,...
93,Motor Mint Distributors Inc.,2003-03-10
94,Signal Collectibles Ltd.,2003-09-12
95,"Double Decker Gift Stores, Ltd",2003-11-14
96,Diecast Collectables,2003-12-09


6. List the customers with the highest revenue in each country.

In [ ]:
q = """
SELECT customerNumber, revenue, country
FROM (
SELECT c.customerNumber, SUM(od.quantityOrdered * od.priceEach) AS revenue, c.country,
ROW_NUMBER() OVER (PARTITION BY c.country ORDER BY SUM(od.quantityOrdered * od.priceEach) DESC) AS rank
FROM orderDetails od
INNER JOIN orders o ON o.orderNumber = od.orderNumber
INNER JOIN customers c ON c.customerNumber = o.customerNumber
GROUP BY c.customerNumber, c.country)
WHERE rank = 1
ORDER BY revenue DESC
"""

pd.read_sql(q, con)

,customerNumber,revenue,country
0,141,820689.54,Spain
1,124,591827.34,USA
2,114,180585.07,Australia
3,119,158573.12,France
4,148,156251.03,Singapore
5,323,154622.08,New Zealand
6,187,148410.09,UK
7,382,137480.07,Austria
8,145,129085.12,Denmark
9,278,127529.69,Italy
